In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

file_path = 'Dataset.csv'
data = pd.read_csv(file_path)

data.fillna(method='ffill', inplace=True)

data['Date'] = pd.to_datetime(data['Date'])

data.set_index('Date', inplace=True)

dataset = data['Close'].values
dataset = dataset.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

sequence_length = 60
X, y = [], []
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

X = np.reshape(X, (X.shape[0], X.shape[1], 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')




<ipython-input-2-c5e690b44120>:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [3]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

train_loss = model.evaluate(X_train, y_train)
test_loss = model.evaluate(X_test, y_test)
print(f'Train Loss: {train_loss}')
print(f'Test Loss: {test_loss}')

Epoch 1/20
133/133 [==============================] - 13s 58ms/step - loss: 0.0061 - val_loss: 2.4422e-04
Epoch 2/20
133/133 [==============================] - 8s 62ms/step - loss: 0.0013 - val_loss: 3.1049e-04
Epoch 3/20
133/133 [==============================] - 8s 61ms/step - loss: 0.0011 - val_loss: 2.4494e-04
Epoch 4/20
133/133 [==============================] - 8s 60ms/step - loss: 0.0012 - val_loss: 2.5423e-04
Epoch 5/20
133/133 [==============================] - 9s 66ms/step - loss: 9.8634e-04 - val_loss: 1.7836e-04
Epoch 6/20
133/133 [==============================] - 7s 52ms/step - loss: 9.9070e-04 - val_loss: 2.3021e-04
Epoch 7/20
133/133 [==============================] - 8s 63ms/step - loss: 9.5348e-04 - val_loss: 5.9663e-04
Epoch 8/20
133/133 [==============================] - 7s 51ms/step - loss: 9.6890e-04 - val_loss: 1.6190e-04
Epoch 9/20
133/133 [==============================] - 8s 63ms/step - loss: 8.8658e-04 - val_loss: 2.3156e-04
Epoch 10/20
133/133 [=============

In [4]:
model.save("LSTM.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.models import load_model
import math

model_path = 'LSTM.h5'
model = load_model(model_path)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

y_train_pred = scaler.inverse_transform(y_train_pred)
y_test_pred = scaler.inverse_transform(y_test_pred)
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

train_mae = mean_absolute_error(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = math.sqrt(train_mse)
train_r2 = r2_score(y_train, y_train_pred)

test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = math.sqrt(test_mse)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Train MAE: {train_mae}')
print(f'Train MSE: {train_mse}')
print(f'Train RMSE: {train_rmse}')
print(f'Train R²: {train_r2}')

print(f'Test MAE: {test_mae}')
print(f'Test MSE: {test_mse}')
print(f'Test RMSE: {test_rmse}')
print(f'Test R²: {test_r2}')


34/34 [==============================] - 1s 15ms/step
Train MAE: 266868.3956866898
Train MSE: 122147778263.23889
Train RMSE: 349496.4638780182
Train R²: -1.3934424370346674
Test MAE: 260533.16660540173
Test MSE: 116868348107.46812
Test RMSE: 341860.1294498499
Test R²: -1.3806145246691952
